# PIA Pacemaker - Python 과정

# Python 실습예제 #4

기초부터 문제까지 차근차근 Build-up

- requests
- beautifulsoup
- selenium


---
### 필수 패키지 설치

In [ ]:
# 필요한 패키지들을 설치합니다. 최초 1회만 실행하면 됩니다.

%pip install requests beautifulsoup4 selenium pandas openpyxl

# 설치가 끝나면 kernel을 "Restart" 합니다.

# 크롬드라이버는 https://wikidocs.net/137919 를 참조하여 설치합니다.


---

### 연습 1 - requests (html 문서 가져오기)

사용 방법  

1. requests 패키지 import
2. 가져올 url을 문자열로 입력하기
3. requests의 get 함수를 이용해 url로부터 html이 담긴 자료 받아오기


In [ ]:
#1
import requests


#2
url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=금메달'


#3
response = requests.get(url)

html_text = response.text


# 일부분을 출력해보자
print(html_text[:1000])


---

### 연습 2 - beautifulsoup (긴 html 문서에서 원하는 것을 가져온다.)

사용방법
1. beautifulsoup 등 패키지 import
2. url로부터 html 가져오기 (연습1과 동일)
3. html을 정리된 형태로 변환시키기 (beautifulsoup 객체)
4. select_one 함수 사용해서 뉴스 제목 가져오기


html에서 가져와야 하는 요소 확인하기:  
. 크롬 브라우저에서 원하는 페이지로 이동 후, F12를 눌러 개발자 도구를 실행합니다.  
. 그 후, 원하는 정보가 담긴 부분에 커서를 올리면 해당 class의 이름을 볼 수 있습니다. ex) a.news_tit 등..


<img src='exercise04_image01.png' width="800px" height="450px"></img>


In [ ]:
#1 
from bs4 import BeautifulSoup as bs
import requests


#2
url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=금메달'
response = requests.get(url)
html_text = response.text


#3
soup = bs(html_text, 'html.parser')


#4
a_tag = soup.select_one('a.news_tit')
title = a_tag.get_text()
href = a_tag.attrs['href']

# 위에서 'news_tit' 클래스를 가진 <a ...> 태그 중 하나를 가져왔다.
# 텍스트와 링크를 출력해보자
print(title)
print(href)


---
### 문제 1
#### 기사 제목 웹스크래핑하기 

1. 원하는 검색어를 검색한 네이버 뉴스 페이지를 가져오고 (html) 
2. 이 페이지에 있는 뉴스 제목과 링크 주소를 전부 출력하세요.

힌트: 
1. 네이버 뉴스 검색 페이지 주소: https://search.naver.com/search.naver?where=news&sm=tab_jum&query=원하는검색어
2. 뉴스 링크에 해당하는 태그.클래스:  a.news_tit


In [ ]:
from bs4 import BeautifulSoup as bs
import requests


url = ...

html_text = ...

soup = ...

a_tags = soup.select(...)

for a in a_tags:
  href = ...
  title = ...
  print(title, '링크:', href)


### 문제 2
#### 원하는 지역의 기온 가져오기

1. input을 통해 지역을 입력받고,
2. 해당 지역의 기온 정보 출력하기

힌트:
1. 특정지역의 네이버 날씨 검색 주소: https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=OO날씨 
2. 기온이 들어있는 태그.클래스:  크롬 개발자 모드를 사용하여 직접 알아내 보세요.  


In [ ]:
from bs4 import BeautifulSoup as bs
import requests


location = input('지역을 입력하세요: ')
url = ...


...

---

### 연습 3 - selenium으로 화면 조작하기

1. selenium 패키지 import
2. 크롬 드라이버 실행
3. 크롬 드라이버로 url 실행
4. 키보드를 이용한 검색 (자동)
5. 마우스 클릭 (자동)


In [ ]:
#1
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time


#2
s = Service('/Users/brix/.chromedriver')  # 내 PC에 저장한 chromedriver의 경로를 줄 것
driver = webdriver.Chrome(service=s)
# 크롬드라이버 사용법 사이트 ( https://jaehyongschool.tistory.com/16 )


#3 
driver.get('https://www.naver.com/')
time.sleep(2)   # 2초간 대기


#4
# 검색어 창을 찾아 search_box 변수에 저장 - css_selector 이용
search_box = driver.find_element(By.CSS_SELECTOR, 'input#query.input_text')

# 검색어 창에 텍스트 입력 (키보드 입력)
search_box.send_keys('대한민국 금메달')
search_box.send_keys(Keys.RETURN)
time.sleep(2)   # 2초간 대기


#5 
# 뉴스 탭 찾아서 변수에 저장 - css_selector 이용
news_link = driver.find_element(By.CSS_SELECTOR, '#lnb > div.lnb_group > div > ul > li:nth-child(8) > a')

# 클릭하기
news_link.click()
time.sleep(2)   # 2초간 대기


---

### 연습 4 - 네이버뉴스 댓글 수집해서 액셀로 저장하기

원본 출처: https://wikidocs.net/151506  


1. selenium, beautifulsoup, pandas 등 필요 패키지 import
2. selenium과 크롬 드라이버 이용해 사이트 방문 (네이버뉴스 기사 페이지의 댓글보기 상태)
3. 더보기 버튼 더이상 안나올 때까지 클릭
4. beautifulsoup 이용해 HTML 요소 모두 파싱
5. pandas 데이터 프레임 생성
6. 엑셀로 저장


<img src='exercise04_image02.png' width="800px" height="450px"></img>

In [ ]:
#1 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import time


#2 
# 크롬 드라이버로 해당 url에 접속
s = Service('/Users/brix/.chromedriver')  # 내 PC에 저장한 chromedriver의 경로를 줄 것
driver = webdriver.Chrome(service=s)

url = 'https://n.news.naver.com/article/comment/092/0002248059'
driver.get(url)
time.sleep(2)


#3
# 더보기가 안뜰 때 까지 계속 클릭 (모든 댓글의 html을 얻기 위함)
while True:
    
# 예외처리 구문 - 더보기 광클하다가 없어서 에러 뜨면 while문을 나감(break)
  try:
    more = driver.find_element(By.CSS_SELECTOR, 'a.u_cbox_btn_more')
    more.click()
    time.sleep(0.2)

  except:
    break


#4
# 본격적인 크롤링 타임

# selenium으로 페이지 전체의 html 문서 받기
html = driver.page_source

# 위에서 받은 html 문서를 bs4 패키지로 parsing
soup = BeautifulSoup(html, 'html.parser')

# 1) 작성자
nicknames = soup.select('span.u_cbox_nick')
list_nicknames = [nickname.text for nickname in nicknames]

# 2) 댓글 시간
datetimes = soup.select('span.u_cbox_date')
list_datetimes = [datetime.text for datetime in datetimes]

# 3) 댓글 내용
contents = soup.select('span.u_cbox_contents') 
list_contents = [content.text for content in contents]

# 4)작성자, 댓글 시간, 내용을 셋트로 취합
list_sum = list(zip(list_nicknames,list_datetimes,list_contents))

# 드라이버 종료
driver.quit()


#5
# 테이블의 첫줄에 들어갈 컬럼명
col = ['작성자', '시간', '내용']

# pandas 데이터 프레임 형태로 가공
df = pd.DataFrame(list_sum, columns=col)


#6
# 데이터 프레임을 엑셀로 저장 (파일명은 'news.xlsx', 시트명은 '뉴스 댓글')
df.to_excel('news.xlsx', sheet_name='뉴스 댓글')

print('저장 완료!!')


---

### 문제 3
#### 구글 학술검색에서 "deep learning construction industry" 검색하여 서지정보 가져오고, 엑셀로 저장하기

1. selenium을 이용해 구글 학술검색으로 이동
2. 검색어 창을 찾아서 "deep learning construction industry" 검색
3. 검색 결과 페이지에서 "인용" 링크 모두 찾기
4. 각 항목에 대해:  
  링크 클릭  
  나타난 팝업 요소에서 MLA 서지정보 저장
5. pandas를 이용해 엑셀로 저장

힌트: 
- 구글 검색창 찾기:  driver.find_element(By.NAME, 'q')
- 인용 버튼 element:  driver.find_element(By.CSS_SELECTOR, '#gs_res_ccl_mid > div > ... > ... ... ')  # 크롬 개발자 모드를 통해 알아내보자
- element에서 텍스트 가져오기:  element.get_attribute("innerHTML")
- 주의: 팝업된 것은 다음 "인용" 버튼 클릭하기 전에 닫아주자. (가려지면 링크 클릭 안됨)


In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import time


#1








#2









#3





#4


















#5










print('저장 완료!!')
